### set the path

In [1]:
%pwd

'/media/kirti/Dev/GenAI/E2E_Job_Recommender/notebooks'

In [2]:
import os
os.chdir('../')
%pwd

'/media/kirti/Dev/GenAI/E2E_Job_Recommender'

### Importing necessary libraries

In [3]:
from src.job_recommender.chains.llm_chain import JobRecommenderChain
from src.job_recommender.schema.output_schema import *

from src.job_recommender.models.llm_model import LLMJobAssistant

### Loading resume

In [4]:
sample_resume = LLMJobAssistant._load_file("sample_resume.txt")

### Chains

#### Resume Summary Chain

In [13]:
resume_summary_chain = JobRecommenderChain(
    schema=ResumeSummary)._get_chain()

In [6]:
# Use the chain as designed: provide file_path and resume_text only
try:
    result = resume_summary_chain.invoke({
        "file_path": "prompts/summary_prompt.txt",
        "resume_text": sample_resume
    })
except Exception as e:
    result = str(e)
# print(result)

Analyze the following resume and provide a brief summary highlighting Skills, Education, Experience and projects and certifications done by the candidate.
Here is the resume:
Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.

Education
----------
M.S. in Computer Science
XYZ University, 2021

B.Tech in Information Technology
ABC Institute of Technology, 2019

Skills
----------
- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL
- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision
- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering
- Tools: Docker, Git, MLflow, AWS, Azure
- Com

In [7]:
print(result.summary)

Kirti Pogra is a results-driven Machine Learning Engineer with 3+ years of experience in designing, developing, and deploying scalable ML solutions. Highlights of the resume include skills in deep learning, data analysis, and model optimization, as well as experience in developing and deploying NLP models, image classification models, and fraud detection systems. The candidate holds a Master's degree in Computer Science and a Bachelor's degree in Information Technology, and has certifications in TensorFlow Developer and AWS Certified Machine Learning – Specialty.


#### Keyword Extraction Chain

In [14]:
keyword_chain = JobRecommenderChain(
    schema=Keywords)._get_chain()

In [9]:
# Use the chain as designed: provide file_path and resume_text only
try:
    result = keyword_chain.invoke({
        "file_path": "prompts/keyword_prompt.txt",
        "resume_text": sample_resume
    })
except Exception as e:
    result = str(e)
# print(result)

Based on the resume , give me the best job titles or keyword to search for finding the best suited job according to resume.
I only want top-5 keywords.
Here is the resume:
Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.

Education
----------
M.S. in Computer Science
XYZ University, 2021

B.Tech in Information Technology
ABC Institute of Technology, 2019

Skills
----------
- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL
- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision
- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering
- Tools: Docker, Git, MLflow, AWS, Azure
- Commun

In [10]:
result

Keywords(keywords=['Machine Learning Engineer', 'Deep Learning', 'NLP', 'Data Engineering', 'Data Analysis'])

#### Skill Gap Analysis Chain

In [15]:
find_skill_gap_chain = JobRecommenderChain(
    schema=FindSkillGap)._get_chain(structured_output=False)

In [12]:
# Use the chain as designed: provide file_path and resume_text only
result = find_skill_gap_chain.invoke({
    "file_path": "prompts/finding_skill_gap_prompt.txt",
    "resume_text": sample_resume
})

Carefully analyze the following resume and identify **missing skills** that are currently in demand for tech jobs (Machine Learning, AI, Data Science).

Only include skills that are NOT already mentioned in the resume.

For each missing skill, return the following:
1. **Skill Name**: (One line)
2. **Description**: A short description of the skill/tool/technology/methodology and its relevance in the current ML/AI job market.
3. **Importance**: Why learning or having this skill would improve the candidate’s career prospects.

Format the response clearly and presentably with bullets or sections for each skill.

Resume:
"""
Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute t

In [17]:
from IPython.display import Markdown
Markdown(result.content)

After analyzing the resume, I've identified some missing skills that are currently in demand for tech jobs in Machine Learning, AI, and Data Science. Here are the missing skills:

**Missing Skills**

### **1. Skill Name:** Transfer Learning

**Description:** Transfer learning is a technique in deep learning where a pre-trained model is fine-tuned on a specific dataset to adapt to a new task. This skill is crucial in the current ML/AI job market, as it allows developers to leverage pre-trained models and reduce training time and data requirements.

**Importance:** Learning transfer learning would improve the candidate's career prospects by enabling them to quickly adapt to new tasks and projects, and by reducing the need for extensive data collection and training.

### **2. Skill Name:** Explainable AI (XAI)

**Description:** Explainable AI is a subfield of AI that focuses on making AI models more transparent and interpretable. This skill is gaining importance in the industry, as organizations need to understand how AI models make predictions and decisions.

**Importance:** Knowing XAI would improve the candidate's career prospects by enabling them to develop trust in AI models and to ensure compliance with regulations like GDPR and HIPAA.

### **3. Skill Name:** Natural Language Processing (NLP) with Attention Mechanisms

**Description:** Attention mechanisms are a key component of NLP models that allow them to focus on specific parts of the input data. This skill is in high demand, as NLP models with attention mechanisms have achieved state-of-the-art results in many applications.

**Importance:** Learning NLP with attention mechanisms would improve the candidate's career prospects by enabling them to develop more accurate and effective NLP models for tasks like language translation, question answering, and text summarization.

### **4. Skill Name:** Cloud-based AutoML

**Description:** Cloud-based AutoML is a set of tools that allows developers to automate the machine learning process, from data preparation to model deployment. This skill is gaining popularity, as organizations seek to streamline their ML workflows and reduce the need for manual intervention.

**Importance:** Knowing cloud-based AutoML would improve the candidate's career prospects by enabling them to automate the ML process and reduce the time and effort required for model development and deployment.

### **5. Skill Name:** Data Visualization with D3.js or Plotly

**Description:** Data visualization is a crucial step in the ML workflow, as it allows developers to understand and communicate insights to stakeholders. D3.js and Plotly are popular data visualization libraries that can help developers create interactive and informative visualizations.

**Importance:** Learning data visualization with D3.js or Plotly would improve the candidate's career prospects by enabling them to effectively communicate insights and

#### Project Ideas Chain

In [16]:
project_idea_chain = JobRecommenderChain(
    schema=ProjectIdeas)._get_chain(structured_output=False)

In [19]:
result = project_idea_chain.invoke({
    "file_path": "prompts/project_ideas_prompt.txt",
    "resume_text": sample_resume
})

Analyze the following resume. Based on the candidate's skills, education, and experience, suggest **5 specific project ideas** that will improve the resume and enhance job opportunities.

Each project idea should include:
1. **Title** of the project
2. A **2-3 line description** explaining what it is and what the candidate will learn or demonstrate by doing it.
3. Align the project with the candidate's background.

Only return a single structured string containing these 5 ideas clearly and concisely.
Here is the resume:
Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.

Education
----------
M.S. in Computer Science
XYZ University, 2021

B.Tech in 

In [17]:
Markdown(result.content)

NameError: name 'Markdown' is not defined

#### Improvement Areas Chain

In [33]:
improvement_areas_chain = JobRecommenderChain(
    schema=ImprovementAreas)._get_chain(structured_output=False)    

In [67]:
result = improvement_areas_chain.invoke({
    "file_path": "prompts/wycd_prompt.txt",
    "resume_text": sample_resume
})

Analyze the provided resume and identify actionable recommendations to enhance its effectiveness for securing more job opportunities.

Instructions:
- Recommendations should be specific and based strictly on the resume content.
- Do NOT rewrite the resume.
- Structure your response in a readable bullet-point format.
- Do NOT include a sample or revised resume.


Here is the resume:
Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.

Education
----------
M.S. in Computer Science
XYZ University, 2021

B.Tech in Information Technology
ABC Institute of Technology, 2019

Skills
----------
- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL
- M

In [68]:
Markdown(result.content)

Here are actionable recommendations to enhance the effectiveness of the resume for securing more job opportunities:

**Objective:**

* Consider rephrasing the objective to make it more specific and concise. For example, instead of "Passionate and results-driven Machine Learning Engineer," try "Experienced Machine Learning Engineer seeking to leverage expertise in deep learning, data analysis, and model optimization to drive business value in innovative projects."

**Education:**

* Consider adding relevant coursework, projects, or academic achievements to demonstrate expertise in machine learning and computer science.
* Consider removing the second degree (B.Tech in Information Technology) as it's not directly relevant to the job seeker's current career goals.

**Skills:**

* Consider adding more specific skills related to machine learning, such as knowledge of transfer learning, reinforcement learning, or attention mechanisms.
* Consider reorganizing the skills section to group similar skills together (e.g., programming languages, machine learning frameworks, data engineering tools).

**Experience:**

* Consider adding more specific metrics to quantify the impact of the projects, such as "Improved customer satisfaction by 25% through personalized product recommendations" instead of "Improved accuracy by 30%".
* Consider highlighting achievements that demonstrate leadership, such as "Led a team to develop and deploy a predictive maintenance model" instead of "Collaborated with data scientists and software engineers".
* Consider removing the "Data Science Intern" role as it's not directly relevant to the job seeker's current experience and expertise.

**Projects:**

* Consider adding more details about the projects, such as the problem statement, methodology, and results. This will help demonstrate the job seeker's skills and expertise.
* Consider removing the "Sentiment Analysis Platform" project as it's not directly related to the job seeker's current experience and expertise.

**Certifications:**

* Consider adding more certifications or relevant training programs to demonstrate expertise and commitment to staying up-to-date with industry developments.
* Consider highlighting the relevance of the certifications to the job seeker's current job role and responsibilities.

Overall, the resume is strong, and the job seeker has a solid foundation to build upon. By refining the objective, education, skills, experience, projects, and certifications sections, the job seeker can make their resume more effective at securing job opportunities.

### Final chain

In [60]:
from langchain.schema.runnable import RunnableParallel
from langchain_core.runnables import RunnableLambda

In [61]:
def create_path(folder_path,file_name):
    return os.path.join(folder_path,file_name)

In [7]:
def safe_dict_access(x, key):
    # print(x,key)
    if not isinstance(x, dict):
        raise TypeError(f"Input to chain must be a dict, got {type(x)}")
    return x[key]


# summary_chain = RunnableLambda(
#     lambda x: resume_summary_chain.invoke({
#         "file_path": create_path(safe_dict_access(x, 'folder_path'), "summary_prompt.txt"),
#         "resume_text": safe_dict_access(x, "resume")
#     })
# )

# keyword_chain = RunnableLambda(
#     lambda x: keywords_chain.invoke({
#         "file_path": create_path(safe_dict_access(x, 'folder_path'), "keyword_prompt.txt"),
#         "summary": safe_dict_access(x, "summary")
#     })
# )


In [31]:
# from langchain_core.runnables import RunnableSequence

In [ ]:
# job_recommendation_chain = RunnableSequence(
#     lambda x: {
#         "folder_path": x["folder_path"],  # store it for later
#         "summary": summary_chain.invoke(x)
#     },
#     lambda x: keyword_chain.invoke({
#         "folder_path": x['folder_path'],
#         "summary": x["summary"]
#     })
# )

In [8]:
print(sample_resume)

Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.

Education
----------
M.S. in Computer Science
XYZ University, 2021

B.Tech in Information Technology
ABC Institute of Technology, 2019

Skills
----------
- Programming: Python, TensorFlow, PyTorch, Scikit-learn, SQL
- Machine Learning: Supervised & Unsupervised Learning, Deep Learning, NLP, Computer Vision
- Data Engineering: Pandas, NumPy, Data Preprocessing, Feature Engineering
- Tools: Docker, Git, MLflow, AWS, Azure
- Communication & Collaboration

Experience
----------
Machine Learning Engineer | Tech Innovators Inc. | June 2021 – Present
- Developed and deployed NLP models for automated resu

In [ ]:
# result = job_recommendation_chain.invoke({
#     "folder_path": "prompts/",
#     "resume": sample_resume,
# })

In [19]:
keywords_chain = RunnableLambda(
    lambda x: keyword_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "keyword_prompt.txt"),
        "resume_text": safe_dict_access(x, "resume")
    })
)

skill_gap_chain = RunnableLambda(
    lambda x: find_skill_gap_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "finding_skill_gap_prompt.txt"),
        "resume_text": safe_dict_access(x,"resume")
    })
)

projects_idea_chain = RunnableLambda(
    lambda x: project_idea_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "project_ideas_prompt.txt"),
        "resume_text": safe_dict_access(x,"resume")
    })
)

improvements_areas_chain = RunnableLambda(
    lambda x: improvement_areas_chain.invoke({
        "file_path": create_path(safe_dict_access(x, 'folder_path'), "wycd_prompt.txt"),
        "resume_text": safe_dict_access(x,"resume")
    })
)

In [20]:
suggestions_chain = RunnableParallel({
        "find_skill_gap": skill_gap_chain, 
        "project_ideas": projects_idea_chain,
        "improvement_areas": improvements_areas_chain
    })     

In [21]:
suggestion_result = suggestions_chain.invoke({
    "folder_path": "prompts/",
    "resume": sample_resume
})

Analyze the following resume. Based on the candidate's skills, education, and experience, suggest **5 specific project ideas** that will improve the resume and enhance job opportunities.

Each project idea should include:
1. **Title** of the project
2. A **2-3 line description** explaining what it is and what the candidate will learn or demonstrate by doing it.
3. Align the project with the candidate's background.

Only return a single structured string containing these 5 ideas clearly and concisely.
Here is the resume:
Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Objective
----------
Passionate and results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Seeking to leverage expertise in deep learning, data analysis, and model optimization to contribute to innovative projects.

Education
----------
M.S. in Computer Science
XYZ University, 2021

B.Tech in 

In [22]:
suggestion_result

{'find_skill_gap': AIMessage(content="After analyzing the resume, I've identified the following missing skills that are currently in demand for tech jobs in Machine Learning, AI, and Data Science:\n\n**Missing Skills:**\n\n**1. Skill Name:** Transfer Learning\n\n**Description:** Transfer learning is a technique where a pre-trained model is fine-tuned on a new dataset to adapt to a specific task. This is particularly useful in scenarios where there is limited labeled data or when the target task is similar to the source task. Transfer learning is widely used in deep learning and has applications in computer vision, natural language processing, and more.\n\n**Importance:** Familiarity with transfer learning would allow the candidate to leverage pre-trained models and adapt them to new tasks, reducing the need for extensive data collection and training from scratch. This skill is highly relevant in today's ML/AI job market, where data efficiency and scalability are crucial.\n\n**2. Skill 

In [24]:
suggestion_result.keys()

dict_keys(['find_skill_gap', 'project_ideas', 'improvement_areas'])

In [ ]:
from IPython.display import Markdown
(Markdown(suggestion_result['find_skill_gap'].content))

After analyzing the resume, I've identified the following missing skills that are currently in demand for tech jobs in Machine Learning, AI, and Data Science:

**Missing Skills:**

**1. Skill Name:** Transfer Learning

**Description:** Transfer learning is a technique where a pre-trained model is fine-tuned on a new dataset to adapt to a specific task. This is particularly useful in scenarios where there is limited labeled data or when the target task is similar to the source task. Transfer learning is widely used in deep learning and has applications in computer vision, natural language processing, and more.

**Importance:** Familiarity with transfer learning would allow the candidate to leverage pre-trained models and adapt them to new tasks, reducing the need for extensive data collection and training from scratch. This skill is highly relevant in today's ML/AI job market, where data efficiency and scalability are crucial.

**2. Skill Name:** Explainable AI (XAI)

**Description:** Explainable AI refers to the practice of designing AI systems that provide insights into their decision-making processes, making them more transparent and accountable. XAI is essential in high-stakes applications, such as healthcare, finance, and law enforcement, where trust and accountability are paramount.

**Importance:** Knowledge of XAI would enable the candidate to design and develop AI systems that are more transparent and explainable, addressing concerns around fairness, accountability, and trust. This skill is gaining importance in the ML/AI job market, as organizations increasingly prioritize transparency and explainability in their AI solutions.

**3. Skill Name:** Reinforcement Learning

**Description:** Reinforcement learning is a type of machine learning that involves training agents to make decisions in complex, dynamic environments. This skill is particularly relevant in applications such as robotics, game playing, and autonomous systems.

**Importance:** Familiarity with reinforcement learning would allow the candidate to develop intelligent agents that can learn from interactions with their environment and make decisions based on rewards and penalties. This skill is highly sought after in the ML/AI job market, as organizations increasingly focus on developing intelligent systems that can interact with and adapt to their environments.

**4. Skill Name:** Natural Language Processing (NLP) with Specialized Libraries

**Description:** NLP is a subfield of AI that focuses on the interaction between computers and humans in natural language. Familiarity with specialized libraries such as spaCy, NLTK, or Stanford CoreNLP would enable the candidate to work with NLP tasks such as text classification, sentiment analysis, and named entity recognition.

**Importance:** Knowledge of NLP with specialized libraries would allow the candidate to develop more sophisticated NLP applications, such as chatbots, language translation systems, and text summarization

In [30]:
Markdown(suggestion_result['project_ideas'].content)

Here are 5 specific project ideas that will improve the resume and enhance job opportunities:

**1.** **Image Segmentation for Medical Diagnosis**: Develop a deep learning-based image segmentation model to identify tumors in medical images, leveraging skills in computer vision and deep learning. This project will demonstrate the candidate's ability to apply machine learning to a real-world problem and work with medical datasets.

**2.** **Natural Language Processing for Text Summarization**: Design and implement a text summarization model using transformer-based architectures, such as BERT or RoBERTa, to summarize long documents or articles. This project will showcase the candidate's expertise in NLP and ability to work with large datasets.

**3.** **Time Series Forecasting for Energy Consumption**: Develop a machine learning model to predict energy consumption patterns using historical data, leveraging skills in data analysis and feature engineering. This project will demonstrate the candidate's ability to work with large datasets and apply machine learning to a real-world problem.

**4.** **Anomaly Detection for Cybersecurity**: Implement an anomaly detection model using one-class SVM or isolation forest to identify unusual network traffic patterns, leveraging skills in supervised learning and data engineering. This project will showcase the candidate's ability to apply machine learning to a cybersecurity context.

**5.** **Explainable AI for Model Interpretability**: Develop an explainable AI model using techniques such as LIME or SHAP to interpret and explain the predictions of a pre-trained model, leveraging skills in deep learning and data analysis. This project will demonstrate the candidate's ability to work with complex models and communicate insights to stakeholders.

These project ideas align with the candidate's background in machine learning, deep learning, and data analysis, and will help to enhance their job opportunities by showcasing their skills and expertise in various areas of machine learning.

In [31]:
Markdown(suggestion_result['improvement_areas'].content)

Here are some actionable recommendations to enhance the effectiveness of the resume:

1. **Quantify achievements**: While the resume provides some numerical data (e.g., "improving accuracy by 30%"), it could benefit from more specific and detailed metrics. For example, instead of "reducing error rate by 15%", consider "reducing error rate from 20% to 5% in just 6 weeks".
2. **Emphasize impact**: While the resume lists various skills and tools, it would be more effective to focus on the impact these skills have had on the projects and companies. For instance, instead of "Built deep learning models for image classification", consider "Built deep learning models for image classification, resulting in a 25% increase in accurate predictions and a 15% reduction in processing time".
3. **Use action verbs consistently**: The resume uses a mix of action verbs (e.g., "Developed", "Built", "Assisted", "Performed"). To improve consistency, consider using a standard set of action verbs throughout the resume, such as "Designed", "Developed", "Implemented", and "Optimized".
4. **Highlight achievements over responsibilities**: While the resume provides a good balance between responsibilities and achievements, it could benefit from more emphasis on the achievements. Consider rephrasing some bullet points to focus on the specific accomplishments rather than the tasks performed.
5. **Organize projects section**: The Projects section is a great way to showcase expertise and creativity. Consider organizing this section by category (e.g., NLP, Computer Vision, etc.) or by the type of project (e.g., competition, personal, etc.) to make it easier for the reader to scan and understand.
6. **Remove redundant information**: The resume mentions both "Machine Learning" and "Deep Learning" skills under the Skills section. Consider removing the redundant information and focusing on the most relevant skills.
7. **Use bullet points consistently**: The resume uses a mix of bullet points and paragraphs. To improve consistency, consider using bullet points throughout the resume to make it easier to scan and read.
8. **Consider adding a summary or summary of qualifications**: The resume could benefit from a brief summary or summary of qualifications at the top to provide a quick overview of the candidate's experience, skills, and achievements.

Here's a revised version of the resume incorporating these recommendations:

Kirti Pogra
Email: kirti.pogra@email.com | Phone: +1-234-567-8901 | LinkedIn: linkedin.com/in/kirtipogra

Summary:
Results-driven Machine Learning Engineer with 3+ years of experience designing, developing, and deploying scalable ML solutions. Proven track

In [32]:
suggestions_chain.get_graph().print_ascii()  # Display the graph of the final chain

            +---------------------------------------------------------------+              
            | Parallel<find_skill_gap,project_ideas,improvement_areas>Input |              
            +---------------------------------------------------------------+              
                          *****              *              *****                          
                    ******                   *                   ******                    
                 ***                         *                         ***                 
   +--------------+                  +--------------+                  +--------------+    
   | _load_prompt |                  | _load_prompt |                  | _load_prompt |    
   +--------------+                  +--------------+                  +--------------+    
           *                                 *                                 *           
           *                                 *                                 *